In [11]:
import spacy
nlp = spacy.load("en_core_web_lg")


In [27]:
import urllib
from string import punctuation
import nltk
import json

In [513]:
import string
import re

In [4]:
import sqlite3
import pandas as pd
from spacy import displacy

In [5]:
con = sqlite3.connect("news_data.sqlite")
cur = con.cursor()

In [6]:
cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
print(cur.fetchall())

[('disasters_reliefweb',), ('search_set',), ('reference_set',), ('chad_narrow_firsthalf_2017_all',), ('chad_narrow_firsthalf_2017_relevance',), ('narrow_2017_relevance',), ('extended_2017_relevance',), ('chad_extended_firsthalf_2017_all',), ('chad_extended_firsthalf_2017_relevance',)]


In [58]:
df = pd.read_sql_query("SELECT * from reference_set", con)

In [59]:
df.reset_index(inplace=True)

In [60]:
df.head()

,index,text,load_date,headline,source,date_published,edition,copyright,length,byline,dateline,other,section,date_formatted
0,0,A major humanitarian crisis is unfolding in Et...,"November 18, 2020",'Full-scale' humanitarian crisis unfolding in ...,defenceWeb,"November 18, 2020",None,Copyright 2020 defenceWeb All Rights Reserved,473,None,None,None,None,18584.0
1,1,Red Cross said it had stepped in to help NHS E...,"January 7, 2017",'This is not Syria or Yemen': Fury as Red Cros...,MailOnline,"January 7, 2017",None,Copyright 2017 Associated Newspapers Ltd. All ...,1470,GARETH DAVIES FOR MAILONLINE,None,None,None,17173.0
2,2,Red Cross said it had stepped in to help NHS E...,"January 7, 2017",'This is not Syria or Yemen': Fury as Red Cros...,MailOnline,"January 7, 2017",None,Copyright 2017 Associated Newspapers Ltd. All ...,1700,GARETH DAVIES FOR MAILONLINE,None,None,None,17173.0
3,3,Very few international news organisations rout...,"October 17, 2018","-City, University of London: New report reveal...",ENP Newswire,"October 17, 2018",None,Copyright 2018 Normans Media Limited All Right...,572,None,None,None,None,17821.0
4,4,The London School of Hygiene & Tropical Medici...,"July 24, 2017",-London School of Hygiene & Tropical Medicine ...,ENP Newswire,"July 24, 2017",None,Copyright 2017 Normans Media Limited All Right...,598,None,None,None,None,17371.0


In [82]:
print(df['headline'][1])
doc = nlp(df['headline'][1])

'This is not Syria or Yemen': Fury as Red Cross calls NHS overcrowding a 'humanitarian crisis' after its worst week in 15 years sees A&Es shut their doors 


In [83]:
print(doc.ents)
for ent in doc.ents:
    print(ent.text, ent.label_)

(Syria, Yemen, Red Cross, NHS, worst week, 15 years)
Syria GPE
Yemen GPE
Red Cross ORG
NHS ORG
worst week DATE
15 years DATE


In [84]:
print(df['text'][1][0:200])
doc1 = nlp(df['text'][1][0:250])

Red Cross said it had stepped in to help NHS England to deal with high demand
After labelling it a 'humanitarian crisis', health service supporters hit back
Former doctor and now Tory MP Sarah Wollast


In [85]:
print(doc1.ents)
for ent in doc1.ents:
    print(ent.text, ent.label_)

(Red Cross, NHS England, Tory, Sarah Wollaston, Syria, Yemen, NHS)
Red Cross ORG
NHS England ORG
Tory NORP
Sarah Wollaston PERSON
Syria GPE
Yemen GPE
NHS ORG


In [16]:
displacy.render(doc, style = "ent", jupyter = True)

In [17]:
ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

In [23]:
lang = "en"
threshold = 0.8

In [24]:
data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "vydltnadaktrbsbirdoubuibinqjyu"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
url = "http://www.wikifier.org/annotate-article"

In [28]:
req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    
with urllib.request.urlopen(req, timeout=60) as f:
    response = f.read()
    response = json.loads(response.decode("utf8"))

In [29]:
def wikifier(text, lang="en", threshold=0.8):
    """Function that fetches entity linking results from wikifier.com API"""
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "vydltnadaktrbsbirdoubuibinqjyu"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    
    with urllib.request.urlopen(req, timeout=60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    
    # Output the annotations.
    results = list()
    
    for annotation in response["annotations"]:
        # Filter out desired entity classes
        if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):

            # Specify entity label
            if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
                label = 'Person'
            elif any([el['enLabel'] in ["company", "enterprise", "business", "organization"] for el in annotation['wikiDataClasses']]):
                label = 'Organization'
            elif any([el['enLabel'] in ["geographic region", "human settlement", "geographic entity", "territorial entity type"] for el in annotation['wikiDataClasses']]):
                label = 'Location'
            else:
                label = None

            results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'], 'label': label,
                            'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    return results

In [247]:
trial = wikifier(df['text'][5], lang="en", threshold=0.8)
#text = df['text'][1][0:200]

In [532]:
df_short = df.iloc[0:100, ]
df_short.head()

,index,text,load_date,headline,source,date_published,edition,copyright,length,byline,dateline,other,section,date_formatted
0,0,A major humanitarian crisis is unfolding in Et...,"November 18, 2020",'Full-scale' humanitarian crisis unfolding in ...,defenceWeb,"November 18, 2020",None,Copyright 2020 defenceWeb All Rights Reserved,473,None,None,None,None,18584.0
1,1,Red Cross said it had stepped in to help NHS E...,"January 7, 2017",'This is not Syria or Yemen': Fury as Red Cros...,MailOnline,"January 7, 2017",None,Copyright 2017 Associated Newspapers Ltd. All ...,1470,GARETH DAVIES FOR MAILONLINE,None,None,None,17173.0
2,2,Red Cross said it had stepped in to help NHS E...,"January 7, 2017",'This is not Syria or Yemen': Fury as Red Cros...,MailOnline,"January 7, 2017",None,Copyright 2017 Associated Newspapers Ltd. All ...,1700,GARETH DAVIES FOR MAILONLINE,None,None,None,17173.0
3,3,Very few international news organisations rout...,"October 17, 2018","-City, University of London: New report reveal...",ENP Newswire,"October 17, 2018",None,Copyright 2018 Normans Media Limited All Right...,572,None,None,None,None,17821.0
4,4,The London School of Hygiene & Tropical Medici...,"July 24, 2017",-London School of Hygiene & Tropical Medicine ...,ENP Newswire,"July 24, 2017",None,Copyright 2017 Normans Media Limited All Right...,598,None,None,None,None,17371.0


In [533]:
entities = {i: wikifier(df_short['headline'][i], lang="en", threshold=0.8) for i in range(len(df_short['headline']))}

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [ ]:
dicto = {i: {} for i in range(len(entities))}

In [ ]:
for i in range(len(entities)):
    for j in range(len(entities[i])):
        dicto[i].update({entities[i][j]['title']: [entities[i][j]['wikiId'], entities[i][j]['label'], entities[i][j]['characters']]})   

In [504]:
dicto

{0: {'Ethiopia': ['Q115', 'Organization', [(46, 53)]]},
 1: {'Syria': ['Q858', 'Organization', [(13, 17)]],
  'Yemen': ['Q805', 'Organization', [(22, 26)]]},
 2: {'Syria': ['Q858', 'Organization', [(13, 17)]],
  'Yemen': ['Q805', 'Organization', [(22, 26)]]},
 3: {'City, University of London': ['Q1094046', 'Organization', [(1, 26)]],
  'University of London': ['Q170027', 'Organization', [(7, 26), (21, 26)]]},
 4: {'London': ['Q84', 'Organization', [(1, 6)]],
  'London School of Hygiene & Tropical Medicine': ['Q1126189',
   'Organization',
   [(1, 24), (1, 44)]]},
 5: {'Novo Nordisk': ['Q818846',
   'Organization',
   [(1, 4), (1, 12), (6, 12), (30, 33), (30, 41), (35, 41)]]},
 6: {'Novo Nordisk': ['Q818846',
   'Organization',
   [(1, 4), (1, 12), (6, 12), (30, 33), (30, 41), (35, 41)]]},
 7: {'International Red Cross and Red Crescent Movement': ['Q7178',
   'Organization',
   [(1, 9)]],
  'Novo Nordisk': ['Q818846', 'Organization', [(15, 18), (15, 26), (20, 26)]]},
 8: {'Africa': ['Q1

In [530]:
for i in range(len(df_short['headline'])):
    doc = nlp(df_short['headline'][i])

    for ent in doc.ents:
        #print(ent.text, ent.label_)
        if ent.label_ in ['GPE', 'ORG', 'LOC']:
            if ent.text not in list(dicto[i].keys()):
                if ent.label_ == 'GPE':
                    dicto[i][ent.text] = ['NA', 'Geopolitical Entity', ['NA']]
                if ent.label_ == 'ORG':    
                    dicto[i][ent.text] = ['NA', 'Organization', ['NA']]
                if ent.label_ == 'LOC':
                    dicto[i][ent.text] = ['NA', 'Location', ['NA']]
                